In [18]:
from demoparser2 import DemoParser
import pandas as pd
parser = DemoParser('g2-vs-faze-m3-nuke.dem')


In [19]:
#gather all player names in case any player throws 0 nades, use as ground truth for final df

player_info = parser.parse_player_info()
player_names = player_info['name'].unique()
player_names

<ArrowExtensionArray>
['karrigan',    'broky',   'frozen',   'm0NESY', 'HeavyGoD',  'malbsMd',
  'huNter-',     'Snax',     'rain',    'EliGE']
Length: 10, dtype: large_string[pyarrow]

In [20]:
#query for each time an hegreanted detonated, and store number of occurences for each player. 
#we also manually inject all of the user_names playing in case someone throws 0 nades, fail safe

frag_exploded = parser.parse_event("hegrenade_detonate")

nades_thrown = frag_exploded['user_name'].value_counts()
total_nades_thrown_df = pd.DataFrame(nades_thrown).reset_index()
total_nades_thrown_df = total_nades_thrown_df.set_index('user_name').reindex(player_names,fill_value=0).reset_index()
total_nades_thrown_df

,user_name,count
0,karrigan,9
1,broky,7
2,frozen,15
3,m0NESY,7
4,HeavyGoD,12
5,malbsMd,8
6,huNter-,10
7,Snax,5
8,rain,9
9,EliGE,7


In [21]:
#query for each time a player damages an enemy with an HE, and sum for each name using groupby. Then clean up and transform into df

player_hurt_df = parser.parse_event("player_hurt")
he_dmg = player_hurt_df[player_hurt_df["weapon"] == "hegrenade"]


dmg_per_player = he_dmg.groupby('attacker_name')['dmg_health'].sum()
dmg_per_player_df = pd.DataFrame(dmg_per_player).reset_index()
dmg_per_player_df.rename(columns={'attacker_name':'user_name'},inplace=True)
dmg_per_player_df

,user_name,dmg_health
0,EliGE,36
1,broky,66
2,frozen,185
3,huNter-,107
4,karrigan,6
5,m0NESY,53
6,malbsMd,129
7,rain,15


In [22]:
#make sure everything looks right

display(dmg_per_player_df)
display(total_nades_thrown_df)

,user_name,dmg_health
0,EliGE,36
1,broky,66
2,frozen,185
3,huNter-,107
4,karrigan,6
5,m0NESY,53
6,malbsMd,129
7,rain,15


,user_name,count
0,karrigan,9
1,broky,7
2,frozen,15
3,m0NESY,7
4,HeavyGoD,12
5,malbsMd,8
6,huNter-,10
7,Snax,5
8,rain,9
9,EliGE,7


In [23]:
#merge on user_name, left merge to make sure all names present. fill the left merged NaN's with 0s. average our values and round. done

nades_merged = pd.merge(total_nades_thrown_df, dmg_per_player_df, on = 'user_name',how='left')
nades_merged['dmg_health'].fillna(0,inplace=True)
nades_merged['avg_dmg_per_HE'] = (nades_merged['dmg_health'] / nades_merged['count']).round(1)
nades_merged

,user_name,count,dmg_health,avg_dmg_per_HE
0,karrigan,9,6.0,0.7
1,broky,7,66.0,9.4
2,frozen,15,185.0,12.3
3,m0NESY,7,53.0,7.6
4,HeavyGoD,12,0.0,0.0
5,malbsMd,8,129.0,16.1
6,huNter-,10,107.0,10.7
7,Snax,5,0.0,0.0
8,rain,9,15.0,1.7
9,EliGE,7,36.0,5.1
